In [1]:
import pandas as pd
import glob
import numpy as np

### GroundWater

In [2]:
# ground-water: unit =mm

In [3]:
gd_dfs = []
for f in glob.glob("../data/groundwater/*"):
    df = pd.read_csv(f, header=1)
    df["loc"] = df.apply(
        lambda x: x["Unnamed: 0"] + "-" + x["Unnamed: 1"], axis=1
    )
    df = df.iloc[1:, 2:]
    df = df.set_index("loc").T
    df.index = pd.to_datetime(df.index)
    gd_dfs.append(df)
df = pd.concat(gd_dfs)
df = df.sort_index()
df = df.replace("-", np.NaN)
df = df.apply(pd.to_numeric)
df = df * 1000
df["date"] = df.index
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [5]:
df.to_csv(r"../data/groundwater.csv", index=False)

### Rainfall

In [9]:
# Rainfall in mm

In [32]:
gd_dfs = []
for f in glob.glob("../data/rainfall/*"):
    df = pd.read_csv(f, header=1)

In [27]:
df=df.replace('-',np.NaN)

In [28]:
df.isna().sum()

Unnamed: 0       0
Unnamed: 1       0
02 Jan 2009    629
03 Jan 2009    629
04 Jan 2009    156
              ... 
28 Dec 2009    629
29 Dec 2009    629
30 Dec 2009    629
31 Dec 2009    588
01 Jan 2010    611
Length: 367, dtype: int64

In [31]:
df.describe()

,Unnamed: 0,Unnamed: 1,02 Jan 2009,03 Jan 2009,04 Jan 2009,05 Jan 2009,06 Jan 2009,07 Jan 2009,08 Jan 2009,09 Jan 2009,...,23 Dec 2009,24 Dec 2009,25 Dec 2009,26 Dec 2009,27 Dec 2009,28 Dec 2009,29 Dec 2009,30 Dec 2009,31 Dec 2009,01 Jan 2010
count,630,630,1,1,474,55,64,1,1,1,...,1,1,1,1,1,1,1,1,42,19
unique,38,620,1,1,385,52,59,1,1,1,...,1,1,1,1,1,1,1,1,40,19
top,UTTAR PRADESH,WEST,Level (m),Level (m),7,4.49,3.77,Level (m),Level (m),Level (m),...,Level (m),Level (m),Level (m),Level (m),Level (m),Level (m),Level (m),Level (m),5.56,4.47
freq,71,2,1,1,5,2,3,1,1,1,...,1,1,1,1,1,1,1,1,2,1
